In [9]:
import cx_Oracle
import pandas as pd 
import sqlite3
import plotly
def FETCH_DATA():
    # Define connection details
    dsn = cx_Oracle.makedsn(
        host = "192.168.1.242",      # Replace with your host IP or domain
        port = 1526,                # Replace with your port
        service_name = "sperpdb"  # Replace with your service name
    )

    # Establish the connection
    connection = cx_Oracle.connect(
        user = "spselect",         # Replace with your username
        password = "select",     # Replace with your password
        dsn = dsn
    )
    cursor = connection.cursor()
    TARGET_COLUMN = ["SC_NO",   # 公司SC號碼
                "KIND_NO",     # 訂單項次
                "CST_PART_NO", # 產品代號
                "PDC_1",       # 生產圖號
                "PDC_2",       # 螺絲規格-1
                "PDC_3",       # 螺絲規格-2
                "PDC_4",       # 螺絲種類
                "PDC_5",       # 變異碼(串華司 鏈帶 頭噴....)
                "MARK_NO",     # 頭記
                "HT_NO",       # 熱處理代號
                "FIN_NO",      # 表面處理代號
                "SALT_SPRAY",  # 鹽測小時數
                "DLV_DATE",    # 訂單交期
                "VEN_DLV_DATE",# 生管交期
                "QTY_PER_CTN", # 每箱數量
                "KEGS",        # 訂單箱數
                "ORDER_QTY1",  # 訂單量(自訂義)
                "ORDER_QTY",   # 訂單量(M)
                "ORDER_WEIG",  # 訂單項次公斤總重
                "PRICE",       # 訂單單價(/M)
                "ORDER_AMT",   # 訂單項次總價
                "PLT_QTY",     # 棧板數
                "COST_PRICE",  # 成本單價(NTD/M)
                "COST_AMT",    # 項次總成本
                "WIR_KIND",    # 線材總類(1022, 1006, 10B21)
                "PDC_1000_WT", # 單重
                "DRAW_NO",     # 圖號
                "SAMPLE_TYPE", # 是否需要樣品
                "SAMPLE_QTY",  # 客戶樣品支數
                "SAMPLE_DLV_DATE",  # 樣品交期
                "CREA_DATE",    # 訂單日期
                "CST_JOB_NO"    # 指定批號
                 ]
    #query = f"SELECT {', '.join(TARGET_COLUMN)} FROM ssl_cst_orde_d"
    query = f"SELECT * FROM ssl_cst_orde_d"
    df = pd.read_sql_query(query, connection)
    df2 = pd.read_sql_query("SELECT SC_NO, ORD_CST_NO FROM ssl_cst_orde_m", connection)
    connection.close()
    return df2.merge(df, on = "SC_NO") 


In [12]:
a = FETCH_DATA()
a = a[pd.to_datetime(a["CREA_DATE"]).dt.date 
                              > pd.to_datetime('2024-01-01').date()]
DATABASE = r"Z:\跨部門\共用資料夾\C. 業務部\詢價統計DB\QUOTATION_DATABASE.db"
def qutoe_data_fetch(customer_code):
    with sqlite3.connect(DATABASE) as connection :
        cursor = connection.cursor()
        query = f"SELECT * FROM CUSTOMER_PRODUCT_SUMMARY WHERE CUSTOMER_CODE like '{customer_code}'"
        df = pd.read_sql_query(query, connection)
    return df
b = qutoe_data_fetch("D09000")
b["QUANTITY"] = b["QUANTITY"].replace("-","100").replace("0","100")

order_sum = a.groupby("CST_PART_NO").agg({
    "ORDER_QTY" : "sum",
    "ORDER_WEIG" : "sum",
    "CST_NO" : "first"
}).reset_index()
order_sum.to_excel("ALL_ITME_ORDER.xlsx")
print(order_sum)
    
    

C:\Users\wesley\AppData\Local\Temp\ipykernel_16472\1935544911.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)
C:\Users\wesley\AppData\Local\Temp\ipykernel_16472\1935544911.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query("SELECT SC_NO, ORD_CST_NO FROM ssl_cst_orde_m", connection)


             CST_PART_NO  ORDER_QTY  ORDER_WEIG  CST_NO
0                   #1QT     1920.0     3552.00  D01200
1                   #2QT      720.0     1814.40  D01200
2                   #3QT      576.0     2724.48  D01200
3                 #610CD     1440.0     1915.20  D01200
4                  #611C     2400.0     3504.00  D01200
...                  ...        ...         ...     ...
1363  ZBG163001006300383      560.8     5304.16  C00600
1364  ZBG163001006300503      300.0     3516.00  C00600
1365  ZBG163001006300633      445.6     6015.60  C00600
1366  ZBG163001006300753      864.0    13262.40  C00600
1367  ZBG163001006300903      100.0     1756.00  C00600

[1368 rows x 4 columns]


In [ ]:
index = 0
aggregated = a.groupby("CST_PART_NO").agg({"ORDER_QTY": "sum", "ORDER_WEIG": "sum"}).reset_index()

# Merge aggregated data back to quotation_sum
quotation_sum = quotation_sum.merge(aggregated, left_on = "PRODUCT_CODE", right_on = "CST_PART_NO")
quotation_sum["ORDER_RATIO"] = quotation_sum["ORDER_QTY"]/quotation_sum["QUANTITY"]
print(quotation_sum)





In [28]:
quotation_sum["ORDER_RATIO"] = quotation_sum["ORDER_QTY"]/quotation_sum["QUANTITY"]
print(quotation_sum)

    QUANTITY    WEIGHT   CST_PART_NO  ORDER_QTY  ORDER_WEIG  ORDER_RATIO
0      780.0    2964.0    CB3BLG158R      320.0      1216.0     0.410256
1     1080.0    2754.0  CB3BLGHL114R      540.0      1377.0     0.500000
2      600.0    1410.0     CBSDQ106R      300.0       705.0     0.500000
3      610.0    2196.0     CBSDQ138R      300.0      1080.0     0.491803
4    16500.0   54450.0     CBSDQ158R     3900.0     12870.0     0.236364
5     3100.0   16756.0    CBSDQL214R     2700.0     14580.0     0.870968
6    50400.0   83664.0      DWC114PR    15120.0     25099.2     0.300000
7    26040.0   51626.4      DWC158PR     7000.0     14420.0     0.268817
8    17150.0   45447.5        DWC2PR     1500.0      3975.0     0.087464
9     6300.0   10962.0     DWC3930PR     1900.0      3477.0     0.301587
10    1000.0    1450.0     DWF3525PR      500.0       725.0     0.500000
11    4990.0   13029.6    DWFSD178PR      400.0      1040.0     0.080160
12     510.0    2244.0    DWFSD238PR      110.0    

In [29]:
quotation_sum.to_excel("D040_summary.xlsx")